In [3]:
%load_ext sql

# Database migration using Liberatii REST API

This notebook contains a tutorial for schema, data migration and verification using Liberatii Migration Framework. To make this notebook executable please set the correct values of the variables in the next cell.

This notebook here is more for helping developers to use the API. For the actual usage, there is a command line utility for running the operations from here.

It migrates Oracle demo HR schema. It expects the schema to be installed in the source Oracle database, and "HR" user is created on the target database.


In [14]:
PG_HOST='demo-g3rqzbqudztzk-pgsql.postgres.database.azure.com'
LGW_HOST='192.168.1.4'
DB='PDBORCL'
USER='HR'
PSWD='hr'
ORACLE_CONN_STR=f'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=192.168.1.6)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME={DB})))'
PG=f'postgresql://{USER}:{PSWD}@{PG_HOST}/PDBORCL'
LGW=f'postgresql://{USER}:{PSWD}@{LGW_HOST}/PDBORCL'
API_PREFIX=f"http://{LGW_HOST}:3000"
ORACLE=f'oracle://{USER}:{PSWD}@{ORACLE_CONN_STR}'
print(f"Open http://{LGW_HOST}:300/api in a browser for API reference")

Open http://192.168.1.4:300/api in a browser for API reference


## Configuration

Now we are going to configure the framework. First, we set Oracle's connection details:

In [3]:
!curl -v {API_PREFIX}/connection -H "Content-Type: application/json" \
   -d '{{"type":"Oracle","connectionString":"{ORACLE_CONN_STR}","user":"{USER}","password":"{PSWD}","id":1}}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /connection HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 178
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 178
< ETag: W/"b2-celWAUSFpweK4ndmD20O4jOkrOc"
< Date: Tue, 08 Aug 2023 15:25:48 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=192.168.1.6)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=PDBORCL)))","user":"HR","password":"hr","id":1}

In [4]:
!curl -v {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{"type":"LGW","host":"{LGW_HOST}","port":5432,"database":"{DB}","user":"{USER}","password":"{PSWD}","id":2}}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /connection HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 103
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 103
< ETag: W/"67-Me2L36c2C4c3Pz0bYS3rAlz2pmA"
< Date: Tue, 08 Aug 2023 15:25:48 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"type":"LGW","host":"192.168.1.4","port":5432,"database":"PDBORCL","user":"HR","password":"hr","id":2}

Next we configure the link to Liberatii Gateway:

In [5]:
!curl -v {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{"type":"LGW","host":"{LGW_HOST}","port":5432,"database":"{DB}","user":"{USER}","password":"{PSWD}", "id":3}}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /connection HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 104
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 103
< ETag: W/"67-bte1lrV6kOGDoBLPVmeZ+2ooXWo"
< Date: Tue, 08 Aug 2023 15:25:48 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"type":"LGW","host":"192.168.1.4","port":5432,"database":"PDBORCL","user":"HR","password":"hr","id":3}

And next some parameters (please, check the reference document to get all the available parameters). Here we set:

* `dataOnePass` - don't use staging files or tables, just transfer all the data from the source to the target
* `user` - list of schemas to transfer between the sources and target target
* `verbose` - log verbosity
* `eraseOnInit` - delete everything from the schema in the init stage, this is useful if we often restart the migration

In [6]:
!curl -v {API_PREFIX}/config -H 'Content-Type: application/json' \
  -d '{{"dataOnePass": true, "users":["{USER}"], "verbose":2, "eraseOnInit":true}}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /config HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 70
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 768
< ETag: W/"300-tNTi9RrxMIfWDXmWXWGRhHiC/Hk"
< Date: Tue, 08 Aug 2023 15:25:48 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"message":"Config has been set successfully.","config":{"dataOnePass":true,"verbose":2,"useCopy":true,"dataIterations":-1,"useWrapper":true,"useNative":false,"useUnlogged":false,"stat":true,"statDB":false,"rowsBuf":1000,"lightCheck":false,"dataChunkSize":-1,"bigTablesFirst":true,"debReverseOrder":false,"cli":true,"rmStagingFiles":true,"parTables":true,"hashType":"murmur","simulateUnsupportedTypes":true,"blobStreams":true,"clobS

## Initialisation

This is an initialisation operation. The framework runs simple assessments and stores the results in the target database. This is an asynchronous operation, it just schedules the operation and exists immediately.


In [7]:
!curl -v {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 2, "stage": "init" }}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 42
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 119
< ETag: W/"77-IbH7ofHMlmIdDR2JdIGFrwucENs"
< Date: Tue, 08 Aug 2023 15:25:48 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Using this operation we can get the current state of the currently running operation:

In [8]:
!curl -v {API_PREFIX}/operation?pager=1 -H 'Content-Type: application/json'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> GET /operation?pager=1 HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 62
< ETag: W/"3e-J8rdTjcvainVzLalq4ZOnK38RTU"
< Date: Tue, 08 Aug 2023 15:25:49 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"title":"init","status":"Running","messages":[],"progress":0}

And the next operatoin is the same as the previous one, except it waits until the current operation ends.

In [9]:
!curl -vX POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation/wait HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 307
< ETag: W/"133-bUUMOgQy4uJakLwtVrAWFnvNMBE"
< Date: Tue, 08 Aug 2023 15:25:52 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"title":"init","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","I","Total"],["TABLE","7","7"],["PROCEDURE","2","2"],["TRIGGER","2","2"],["SEQUENCE","3","3"],["INDEX","11","11"],["VIEW","1","1"],["^+Total:^:","26","^+26^"]]}],"progress":0}

## Schema migration

Now, after `init` stage is done, we have all the necessary information for migrating the database schema. It's stored in the target DB and we can run various queries on it. 

For example, we can query the number of objects of each type by running with the query:


In [10]:
%%sql {PG}
select count(*), type, stage, error from dbt.migration_objects group by type, stage, error

6 rows affected.


count,type,stage,error
3,SEQUENCE,I,None
11,INDEX,I,None
1,VIEW,I,None
2,PROCEDURE,I,None
7,TABLE,I,None
2,TRIGGER,I,None


If there were any errors during the execution of any stage, they will be displayed in `error` stage. The errors can be manually fixed by modifying `ddl1`, `ddl2` columns of `dbt.migration_objects` or by adding some runtime objects. 

When errors are fixed we don't need to run the whole migration from the beginning. It's enough to reset only the problematic objects' `stage` field and restart the stage. Only those objects migration will be re-run.

Now starting the schema migration, with the same parameters as with `init` stage, but with `schema` as the stage name.

The schema migration doesn't migrate constraints, indexes and triggers. They are migrated in `constraints` stage. This is done to make data migration run faster.

We can also run several instances of each operation. In this case, the migration will run in parallel this way speeding it up significantly. Even a single table data migration can be parallelised.


In [11]:
!curl -v {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 2, "stage": "schema" }}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 44
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 119
< ETag: W/"77-IbH7ofHMlmIdDR2JdIGFrwucENs"
< Date: Tue, 08 Aug 2023 15:25:53 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Again, waiting until this task is finished:


In [12]:
!curl -vX POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation/wait HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 353
< ETag: W/"161-2IY1nkWOzapKYoESO9+DROYM0bI"
< Date: Tue, 08 Aug 2023 15:25:54 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"title":"schema","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","I","D","Total"],["TABLE","","^g7^","7"],["VIEW","","^g1^","1"],["TRIGGER","2","","2"],["INDEX","11","","11"],["SEQUENCE","","^g3^","3"],["PROCEDURE","","^g2^","2"],["^+Total:^:","13","^g^+13^","^+26^"]]}],"progress":1}

In the `init` stage the framework also queries the sizes of each table. Using them we can get the most optimal strategy for copying the data.

So let's see the sizes:


In [13]:
%%sql {PG}
select name, pg_size_pretty(data_size), stage from dbt.migration_objects where type = 'TABLE' order by data_size desc

7 rows affected.


name,pg_size_pretty,stage
JOB_HISTORY,64 kB,D
REGIONS,64 kB,D
DEPARTMENTS,64 kB,D
LOCATIONS,64 kB,D
EMPLOYEES,64 kB,D
JOBS,64 kB,D
COUNTRIES,0 bytes,D



By analysing the sizes for different tables we can apply different strategies for different tables. But size the sizes are quire small for this schema we just use the default strategy everywhere.

Now starting the data migration operation, as all the operations before, but with `data` as `stage` field value:


In [14]:
!curl -v {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 2, "stage": "data" }}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 42
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 119
< ETag: W/"77-IbH7ofHMlmIdDR2JdIGFrwucENs"
< Date: Tue, 08 Aug 2023 15:25:55 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Awaiting the operation to be completed:

In [15]:
!curl -vX POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation/wait HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 381
< ETag: W/"17d-zJDFU1APqibjOfrYY88vn8GfWPo"
< Date: Tue, 08 Aug 2023 15:25:56 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"title":"data","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","I","D","R","Total"],["VIEW","","^g1^","","1"],["TRIGGER","2","","","2"],["INDEX","11","","","11"],["TABLE","","","^g7^","7"],["SEQUENCE","","^g3^","","3"],["PROCEDURE","","^g2^","","2"],["^+Total:^:","13","^g^+6^","^g^+7^","^+26^"]]}],"progress":1}

## Constraints
This stage adds constraints, indexes and triggers to the data we've moved in the previous step:


In [16]:
!curl -v {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 2, "stage": "constraints" }}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 49
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 119
< ETag: W/"77-IbH7ofHMlmIdDR2JdIGFrwucENs"
< Date: Tue, 08 Aug 2023 15:25:56 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Awaiting the operation to be completed:

In [17]:
!curl -vX POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation/wait HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 402
< ETag: W/"192-sEzS2kS/xiucs14rJb9sOeSSPGk"
< Date: Tue, 08 Aug 2023 15:25:58 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"title":"constrants","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","D","R","d","Total"],["VIEW","^g1^","","","1"],["TRIGGER","^g2^","","","2"],["INDEX","^g11^","","","11"],["TABLE","","^g3^","^g4^","7"],["SEQUENCE","^g3^","","","3"],["PROCEDURE","^g2^","","","2"],["^+Total:^:","^g^+19^","^g^+3^","^g^+4^","^+26^"]]}],"progress":1}

Now we can verify the data in both databases are the same, by comparing each column hash sums:

In [18]:
!curl -v {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 2, "stage": "check" }}'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 43
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 119
< ETag: W/"77-IbH7ofHMlmIdDR2JdIGFrwucENs"
< Date: Tue, 08 Aug 2023 15:25:58 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

In [19]:
!curl -vX POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

*   Trying 192.168.1.4:3000...
* Connected to 192.168.1.4 (192.168.1.4) port 3000 (#0)
> POST /operation/wait HTTP/1.1
> Host: 192.168.1.4:3000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< X-Powered-By: Express
< Content-Type: application/json; charset=utf-8
< Content-Length: 362
< ETag: W/"16a-A0cKXCoF9Nse7WpqoirJ2gfWjlo"
< Date: Tue, 08 Aug 2023 15:26:00 GMT
< Connection: keep-alive
< Keep-Alive: timeout=5
< 
* Connection #0 to host 192.168.1.4 left intact
{"title":"check","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","D","K","Total"],["VIEW","^g1^","","1"],["TRIGGER","^g2^","","2"],["INDEX","^g11^","","11"],["SEQUENCE","^g3^","","3"],["TABLE","","^g7^","7"],["PROCEDURE","^g2^","","2"],["^+Total:^:","^g^+19^","^g^+7^","^+26^"]]}],"progress":1}

This is it, now the whole database is migrated. Please see the guide document for the information about different migration options.

## Running queries

Let's now run a few queries in the both databases to see the results are identical. There is a `checksql` stage to do this automatically, but for the demo purposes we do this manually.

So first we check the databases are indeed Oracle nad PostgreSQL

In [6]:
%%sql {ORACLE}
select banner from v$version

0 rows affected.


banner
Oracle Database 12c Enterprise Edition Release 12.1.0.2.0 - 64bit Production
PL/SQL Release 12.1.0.2.0 - Production
CORE 12.1.0.2.0 Production
TNS for 64-bit Windows: Version 12.1.0.2.0 - Production
NLSRTL Version 12.1.0.2.0 - Production


In [7]:
%%sql {LGW}
select banner from v$version

1 rows affected.


BANNER
"PostgreSQL 14.7 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit"


The output value on PostgreSQL can be configured to output exactly the same like Oracle if the application uses the data for some business logic selection.

Now let's run the same query in both databases.

In [10]:
QUERY="""
SELECT first_name  || ' ' ||  last_name full_name,
       salary + 
       NVL  (commission_pct, 0) sal_com, 
       DECODE (NVL(e.department_id, -3), 60, d.department_name, 
                                50, d.department_name, 
                                30, d.department_name, 
                                -3, 'UNKNOWN',
               'OTHER') dep,
       TO_CHAR(e.hire_date) hire_date
  FROM employees e, departments d
 WHERE d.department_id(+) = e.department_id
   AND e.last_name like 'G%'
ORDER BY first_name  ||  ' '  ||  last_name
"""

First running it on Oracle:

In [12]:
%%sql {ORACLE} 

{QUERY}


0 rows affected.


full_name,sal_com,dep,hire_date
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600,Shipping,13-JAN-08
Girard Geoni,2800,Shipping,03-FEB-08
Ki Gee,2400,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008,OTHER,17-AUG-02
Timothy Gates,2900,Shipping,11-JUL-06
William Gietz,8300,OTHER,07-JUN-02


And now running absolutely the same query but on PostgreSQL via Liberatii Gateway:

In [13]:
%%sql {LGW} 

{QUERY}

8 rows affected.


FULL_NAME,SAL_COM,DEP,HIRE_DATE
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600.00,Shipping,13-JAN-08
Girard Geoni,2800.00,Shipping,03-FEB-08
Ki Gee,2400.00,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008.00,OTHER,17-AUG-02
Timothy Gates,2900.00,Shipping,11-JUL-06
William Gietz,8300.00,OTHER,07-JUN-02


Here you may see the results are identical. 

The next steps are:

* Synchronise the databases using CDC so we avoid downtimes on switchover
* Replay workloads on a sandbox Oracle database and PostgreSQL via Liberatii Gateway, for testing correctness and performance
